<a href="https://colab.research.google.com/github/mariosilvestri3/capstone-objectives-1-2-3/blob/main/objective-2-data-preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TelCo Customer Churn - Capstone for Mario Silvestri
## Objective 2 - Data Preparation
This notebook is the full data preparation process and final deliverable for Objective 2 - Data Preparation.

### Load Dataset from Datasource

In [71]:
import pandas as pd
import plotly.express as px

churn_data_file = 'https://raw.githubusercontent.com/mariosilvestri3/capstone/bf1edf1b098c856b49eaf32e6832c91ff891bd74/data/customer-churn-raw.csv'
df = pd.read_csv(churn_data_file)

### Examine and Understand Dataset

In [72]:
df.head()

,LoyaltyID,Customer ID,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
0,318537,7590-VHVEG,No,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,152148,5575-GNVDE,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,326527,3668-QPYBK,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,845894,7795-CFOCW,No,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,503388,9237-HQITU,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [73]:
df.tail()

,LoyaltyID,Customer ID,Senior Citizen,Partner,Dependents,Tenure,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn
7038,810338,6840-RESVB,No,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,230811,2234-XADUH,No,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,155157,4801-JZAZL,No,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,731782,8361-LTMKD,Yes,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes
7042,353947,3186-AJIEK,No,No,No,66,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


In [74]:
df.columns

Index(['LoyaltyID', 'Customer ID', 'Senior Citizen', 'Partner', 'Dependents',
       'Tenure', 'Phone Service', 'Multiple Lines', 'Internet Service',
       'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
       'Streaming TV', 'Streaming Movies', 'Contract', 'Paperless Billing',
       'Payment Method', 'Monthly Charges', 'Total Charges', 'Churn'],
      dtype='object')

In [75]:
df.shape

(7043, 21)

#### The dataset has 7,043 rows and 21 columns. Churn column is our dependant variable. 

#### LoyaltyID and Customer ID both appear to be unique customer identifiers. Let's verify.

In [76]:
df.iloc[:, 0:2].nunique()

LoyaltyID      7021
Customer ID    7043
dtype: int64

##### LoyaltyID is not a unique identifier because it is repeated. Consulting with a subject matter expert on this datasource would provide more information as to why some customers share a Loyalty ID.


### Describe Data
|Attribute|Statistical Type|
|---|---|
|LoyaltyID| Non-unique Identifier|
|Customer ID|Unique Identifier
|Senior Citizen|Nominal
|Partner|Nominal
|Tenure|Ratio|
|Phone Service|Nominal|
|Multiple Lines|Nominal|
|Internet Service|Nominal|
|Online Security|Nominal|
|Online Backup|Nominal|
|Device Protection|Nominal|
|Tech Support|Nominal|
|Streaming TV|Nominal|
|Streaming Movies|Nominal|
|Contract|Nominal|
|Paperless Billing|Nominal|
|Payment Method|Nominal|
|Monthly Charges|Ratio|
|Total Charges|Ratio|
|Churn|Nominal|

### Quality Assurance Testing - Verify Data Quality

### Feature Engineering, Data Parsing, Cleaning, and Wrangling

#### Drop LoyaltyID, Customer ID, and Total Charges rows.
Once each row has been verified as a unique entry, these rows are no longer needed to continue with descriptive and predictive analysis.

In [77]:
df = df.drop(columns=['LoyaltyID', 'Customer ID', 'Total Charges'])
df.columns

Index(['Senior Citizen', 'Partner', 'Dependents', 'Tenure', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Churn'],
      dtype='object')

### Analyze Data
Measure churn against various customer attributes to uncover insights and patterns.

#### Analyze Senior Citizens

In [78]:
slice_df = df[['Churn', 'Senior Citizen']]
fig = px.histogram(slice_df, x='Senior Citizen', barmode='stack', color='Churn')
fig.show()

Findings: Senior citizens are slightly more likely to churn and represent a small portion of our dataset.

#### Analyze Contract Term Length

In [79]:
slice_df = df[['Churn', 'Contract']]
fig = px.histogram(slice_df, x='Contract', barmode='stack', color='Churn')
fig.show()

Findings: Month-to-month customers are significantly more likely to churn and represent a large portion of our customers.

### Analyze Monthly Charges

In [80]:
slice_df = df[['Churn', 'Monthly Charges']]
fig = px.violin(slice_df, x='Churn', y='Monthly Charges', color='Churn')
fig.show()

In [81]:
fig = px.histogram(slice_df, x='Monthly Charges', color='Churn', nbins=12)
fig.show()

Findings: Customers with monthly charges ranging from \$70 to \$105 have a higher chance of churning. Customers with monthly bills under \$30 are less likely to churn.

### Multiple Linear Regression Predictive Analysis with Dummy Variable Feature Engineering
Analyzing the resulting correllation coefficients will provide a broad overview of each feature's significance in predicting churn.

In [82]:
from sklearn.linear_model import LinearRegression
X = df.drop(columns=['Churn'])
# Feature Engineering is used here to encode categorical data as numerical data.
X = pd.get_dummies(X, columns=['Senior Citizen', 'Partner', 'Dependents',
                               'Phone Service', 'Multiple Lines',
                               'Internet Service', 'Online Security', 
                               'Online Backup', 'Device Protection', 
                               'Tech Support', 'Streaming TV',
                               'Streaming Movies', 'Contract',
                               'Paperless Billing', 'Payment Method',], 
                   prefix_sep=' = ', drop_first=True)
y = df['Churn']
y = pd.get_dummies(y)

model = LinearRegression(n_jobs=10)
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=10, normalize=False)

In [83]:
colors = ['Positive' if c > 0 else 'Negative' for c in model.coef_[1]]

fig = px.bar(y=X.columns, x=model.coef_[1], color=colors, 
             color_discrete_map={'Negative': 'blue', 'Positive': 'red'}, 
             title="Correlation of feature to churn",
             orientation='h', labels=(dict(x='Correlation', y='Feature')),
             height=800, width=800)
fig.update_yaxes(categoryorder='total ascending', type='category')
fig.update_layout(legend=dict(orientation='h', yanchor='top', y=1.05, x=1, xanchor='right'))
fig.show()

Findings:  

*   Contract term, internet service, payment method, and multiple lines significantly predicts churn.
*   Some internet subservices such as streaming, tech support, online security, and others are significant predictors.
*   Monthly charges, phone service, partner, and tenure are not significant predictors; These can be removed from the feature set.

### Confirm Correlations - Analyze Internet Service Type and Payment Method

In [84]:
slice_df = df[['Churn', 'Internet Service']]
fig = px.histogram(slice_df, x='Internet Service', barmode='stack', color='Churn')
fig.show()

In [85]:
slice_df = df[['Churn', 'Payment Method']]
fig = px.histogram(slice_df, x='Payment Method', barmode='stack', color='Churn')
fig.show()

Findings: It is clear that fiber optic and electronic check customers are more likely to churn.

### Data Wrangling - Feature Selection
Remove uncorrelated features.

In [87]:
df = df.drop(columns=['Monthly Charges', 'Phone Service', 'Partner', 'Tenure'])
df.head()

,Senior Citizen,Dependents,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Churn
0,No,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,No
1,No,No,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,No
2,No,No,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,Yes
3,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),No
4,No,No,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,Yes


### Final Dataset